In [1]:
!pip install openvino-dev

  Obtaining dependency information for openvino-dev from https://files.pythonhosted.org/packages/86/65/76ce426ac1e1d7db560187440b307be10af108a92d28e3e1842e388989db/openvino_dev-2023.1.0-12185-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for openvino-telemetry>=2022.1.0 from https://files.pythonhosted.org/packages/18/7e/656e52224ef145abed0c92535c786181b933f87323fc88824e947f611119/openvino_telemetry-2023.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for texttable>=1.6.3 from https://files.pythonhosted.org/packages/24/99/4772b8e00a136f3e01236de33b0efda31ee7077

In [1]:
import os
import time
from transformers import ViTImageProcessor
import torch
from PIL import Image
from sklearn.metrics import accuracy_score
import numpy
from openvino.runtime import serialize
from openvino.tools import mo
from openvino.runtime import Core

In [2]:
# Путь к нашему ViT, который будем конвертировать в OpenVINO.
path_to_onnxmodel = "vit.onnx"

# Конвертируем ViT в OpenVINO формат.
openvino_model = mo.convert_model(path_to_onnxmodel)
serialize(openvino_model, 'vit_openvino.xml')

In [3]:
# Функция для запуска конвертированной в OpenVINO модели. 
def openvino_model_use(compiled_openvino_model, img):
    
    model_id2label=  {0: "cats", 1: "dogs"}
    
    out_openvino = compiled_openvino_model(img['pixel_values'].numpy())[0][0]

    predicted_label = out_openvino.argmax(-1).item()

    return model_id2label[predicted_label]

In [4]:
# Путь к тестовым картинкам.
path_to_images = "data/"

path_to_model = "weights/my_model"

images_list = os.listdir(path_to_images)

# Преобразование картинок в тензоры
extractor = ViTImageProcessor.from_pretrained(path_to_model)

In [5]:
# Загрузим нашу OpenVINO модель.
core = Core()
vit_openvino_model = core.read_model('vit_openvino.xml')
compiled_openvino_model = core.compile_model(vit_openvino_model, "CPU")

In [6]:
# Проверим размер ONNX модели.
onnx_file_size = os.path.getsize("vit.onnx")/(1024**2)

print("Размер модели в ONNX ", onnx_file_size, " мегабайт.")

Размер модели в ONNX  327.5526752471924  мегабайт.


In [7]:
# Проверим размер OpenVINO модели.
openvino_file_size = os.path.getsize("vit_openvino.bin")/(1024**2)

print("Размер модели в OpenVINO ", openvino_file_size, " мегабайт.")

Размер модели в OpenVINO  327.3019828796387  мегабайт.


In [9]:
# Запустим тест нашей OpenVINO модели.

start_time = time.time()

# Собака 1, кошка 0.
target_list = []
predict_list = []

for element in images_list:

    image = Image.open(path_to_images + element, mode='r', formats=None)

    inputs = extractor(image, return_tensors="pt")
    predict = openvino_model_use(compiled_openvino_model, inputs)
    target = element[:element.find(".")]

    if target == "dog":
        label = 1
    else:
        label = 0

    target_list.append(label)

    if predict == "dogs":
        pr = 1
    else:
        pr = 0

    predict_list.append(pr)

end_time = time.time()

acc = accuracy_score(target_list, predict_list)
print("Точность сконвертированной в OpenVINO модели= ", acc)
print("Время обработки изображений сконвертированной в OpenVINO моделью = ", end_time-start_time, " секунд")
print("Скорость обработки изображений у сконвертированной в OpenVINO модели составила  ", len(images_list)/(end_time-start_time), " картинок в секунду")

Точность сконвертированной в OpenVINO модели=  0.9875
Время обработки изображений сконвертированной в OpenVINO моделью =  13.248905658721924  секунд
Скорость обработки изображений у сконвертированной в OpenVINO модели составила   12.076469115369537  картинок в секунду
